In [1]:
# This script creates a Google Cloud Storage Bucket, BigQuery Dataset (Data Warehouse), and BigQuery Tables

In [11]:
import os
from google.cloud import storage
from google.cloud import bigquery
from google.oauth2 import service_account

In [12]:
# Key to Google Cloud Account Credentials
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = 'Key/pragmatic-bongo-404116-e2d94f71da27.json'

#### Create Google Cloud Storage Bucket

In [13]:
def create_bucket(bucket_name, project_id):
    """Creates a new bucket in a specific project."""
    storage_client = storage.Client(project=project_id)
    bucket = storage_client.bucket(bucket_name)
    new_bucket = storage_client.create_bucket(bucket, project=project_id)
    print(f"Bucket {new_bucket.name} created")
# Create a new bucket
project_id = 'pragmatic-bongo-404116'
bucket_name = 'cis4400_group_project'
try:
    create_bucket(bucket_name, project_id)
except Exception as e:
    print(f"{e}")

409 POST https://storage.googleapis.com/storage/v1/b?project=pragmatic-bongo-404116&prettyPrint=false: Your previous request to create the named bucket succeeded and you already own it.


#### Create BigQuery Dataset

In [14]:
def create_bigquery_dataset(project_id, dataset_name):
    """Creates a BigQuery dataset."""
    bigquery_client = bigquery.Client(project=project_id)
    dataset_id = f"{project_id}.{dataset_name}"
    dataset = bigquery.Dataset(dataset_id)
    dataset.location = "US"
    bigquery_client.create_dataset(dataset)
    print(f"Dataset {dataset_id} created.")

project_id = 'pragmatic-bongo-404116'
dataset_name = 'ownership_payment'
try:
    create_bigquery_dataset(project_id, dataset_name)
except Exception as e:
    print(f"{e}")

409 POST https://bigquery.googleapis.com/bigquery/v2/projects/pragmatic-bongo-404116/datasets?prettyPrint=false: Already Exists: Dataset pragmatic-bongo-404116:ownership_payment


#### Create Tables in BigQuery

In [15]:
# Get the path to the service account key file from the environment variable
service_account_path = os.environ.get('GOOGLE_APPLICATION_CREDENTIALS')

# Set your Google Cloud credentials using the environment variable
credentials = service_account.Credentials.from_service_account_file(service_account_path)
# Initialize a BigQuery client
client = bigquery.Client(credentials=credentials, project=credentials.project_id)

# Define your dataset and table names
dataset_name = 'ownership_payment'
fact_table_name = 'Payment_Fact'
location_dim_table_name = 'DimLocation'
physician_dim_table_name = 'DimPhysician'
manufacturer_dim_table_name = 'DimManufacturerOrGpaPayer'
date_dim_table_name = 'DimDate'

# Create the dataset
dataset_ref = client.dataset(dataset_name)
client.get_dataset(dataset_ref)

# Define the schema for the fact table
fact_table_schema = [
    bigquery.SchemaField('RECORD_ID', 'INTEGER', mode='REQUIRED'),
    bigquery.SchemaField('LOCATION_ID', 'STRING'),
    bigquery.SchemaField('PHYSICIAN_PROFILE_ID', 'INTEGER'),
    bigquery.SchemaField('APPLICABLE_MANUFACTURER_OR_APPLICABLE_GPO_MAKING_PAYMENT_ID', 'INTEGER'),
    bigquery.SchemaField('DATE_ID', 'STRING'),
    bigquery.SchemaField('VALUE_OF_INTEREST', 'FLOAT'),
    bigquery.SchemaField('TOTAL_AMOUNT_INVESTED_USDOLLARS', 'FLOAT'),
    bigquery.SchemaField('DISPUTE_STATUS_FOR_PUBLICATION', 'BOOL') 
]

# DEFINE SCHEMA FOR REST OF DIMENSION TABLES AND FACT TABLE BASED ON DATA MODEL
location_dim_table_schema = [
    bigquery.SchemaField('LOCATION_ID', 'STRING', mode='REQUIRED'),
    bigquery.SchemaField('RECIPIENT_PRIMARY_BUSINESS_STREET_ADDRESS', 'STRING'),
    bigquery.SchemaField('RECIPIENT_CITY', 'STRING'),
    bigquery.SchemaField('RECIPIENT_STATE', 'STRING'),
    bigquery.SchemaField('RECIPIENT_ZIP_CODE', 'STRING'),
    bigquery.SchemaField('RECIPIENT_COUNTRY', 'STRING'),
    bigquery.SchemaField('RECIPIENT_PROVINCE', 'STRING'),
    bigquery.SchemaField('RECIPIENT_POSTAL_CODE', 'STRING')
]
physician_dim_table_schema = [
    bigquery.SchemaField('PHYSICIAN_PROFILE_ID', 'STRING', mode='REQUIRED'),
    bigquery.SchemaField('PHYSICIAN_FIRSTNAME', 'STRING'),
    bigquery.SchemaField('PHYSICIAN_MIDDLENAME', 'STRING'),
    bigquery.SchemaField('PHYSICIAN_LASTNAME', 'STRING'),
    bigquery.SchemaField('PHYSICIAN_NPI', 'STRING'),
    bigquery.SchemaField('PHYSICIAN_PRIMARY_TYPE', 'STRING'),
    bigquery.SchemaField('PHYSICIAN_SPECIALTY', 'STRING'),
    bigquery.SchemaField('PHYSICIAN_NAME_SUFFIX', 'STRING'),
    bigquery.SchemaField('INTEREST_HELD_BY_PHYSICIAN_OR_AN_IMMEDIATE_FAMILY_MEMBER', 'STRING')

]
manufacturer_dim_table_schema = [
    bigquery.SchemaField('APPLICABLE_MANUFACTURER_OR_APPLICABLE_GPO_MAKING_PAYMENT_ID', 'STRING', mode='REQUIRED'),
    bigquery.SchemaField('SUBMITTING_APPLICABLE_MANUFACTURER_OR_APPLICABLE_GPO_NAME', 'STRING'),
    bigquery.SchemaField('APPLICABLE_MANUFACTURER_OR_APPLICABLE_GPO_MAKING_PAYMENT_NAME', 'STRING'),
    bigquery.SchemaField('APPLICABLE_MANUFACTURER_OR_APPLICABLE_GPO_MAKING_PAYMENT_STATE', 'STRING'),
    bigquery.SchemaField('APPLICABLE_MANUFACTURER_OR_APPLICABLE_GPO_MAKING_PAYMENT_COUNTRY', 'STRING')

]
date_dim_table_schema = [
    bigquery.SchemaField('DATE_ID', 'STRING', mode = 'REQUIRED'),
    bigquery.SchemaField('PAYMENT_PUBLICATION_DATE', 'DATE'),
    bigquery.SchemaField('PROGRAM_YEAR', 'INTEGER')
]


# Create the tables
fact_table_ref = dataset_ref.table(fact_table_name)
try:
    client.get_table(fact_table_ref)
    print(f"Table {fact_table_name} already exists in the dataset {dataset_name}.")
except:
    fact_table = bigquery.Table(fact_table_ref, schema=fact_table_schema)
    client.create_table(fact_table)
    print(f"{fact_table_name} Created")
    
location_dim_table_ref = dataset_ref.table(location_dim_table_name)
try:
    client.get_table(location_dim_table_ref)
    print(f"Table {location_dim_table_name} already exists in the dataset {dataset_name}.")
except:
    location_dim_table = bigquery.Table(location_dim_table_ref, schema=location_dim_table_schema)
    client.create_table(location_dim_table)
    print(f"{location_dim_table_name} Created")
    
physician_dim_table_ref = dataset_ref.table(physician_dim_table_name)
try:
    client.get_table(physician_dim_table_ref)
    print(f"Table {physician_dim_table_name} already exists in the dataset {dataset_name}.")
except:
    physician_dim_table = bigquery.Table(physician_dim_table_ref, schema=physician_dim_table_schema)
    client.create_table(physician_dim_table)
    print(f"{physician_dim_table_name} Created")
    
manufacturer_dim_table_ref = dataset_ref.table(manufacturer_dim_table_name)
try:
    client.get_table(manufacturer_dim_table_ref)
    print(f"Table {manufacturer_dim_table_name} already exists in the dataset {dataset_name}.")
except:
    manufacturer_dim_table = bigquery.Table(manufacturer_dim_table_ref, schema=manufacturer_dim_table_schema)
    client.create_table(manufacturer_dim_table)
    print(f"{manufacturer_dim_table_name} Created")
    
date_dim_table_ref = dataset_ref.table(date_dim_table_name)
try:
    client.get_table(date_dim_table_ref)
    print(f"Table {date_dim_table_name} already exists in the dataset {dataset_name}.")
except:
    date_dim_table = bigquery.Table(date_dim_table_ref, schema=date_dim_table_schema)
    client.create_table(date_dim_table)
    print(f"{date_dim_table_name} Created")

Table Payment_Fact already exists in the dataset ownership_payment.
Table DimLocation already exists in the dataset ownership_payment.
Table DimPhysician already exists in the dataset ownership_payment.
Table DimManufacturerOrGpaPayer already exists in the dataset ownership_payment.
Table DimDate already exists in the dataset ownership_payment.
